# Prerrequisites

Installing Spark


---



In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.2.0'

Creating tunnel</br>
**To Check the Spark UI, open the URL printed by running the above command : https://######/jobs/, /SQL/**


In [5]:
 from google.colab.output import eval_js
 print(eval_js("google.colab.kernel.proxyPort(4040)") + "jobs/")

https://qagmjachch-496ff2e9c6d22116-4040-colab.googleusercontent.com/jobs/


# Download Datasets

In [6]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/species.csv -P /dataset
!ls /dataset

bank.csv  characters.csv  planets.csv  species.csv  vehicles.csv


# Windows Partitioning

---



## Example 1

In [7]:
!head /dataset/bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

Reading data from `bank.csv` file to a DataFrame

In [8]:
from pyspark.sql.functions import *

bank_df = spark.read.format("csv") \
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/bank.csv")

In [9]:
bank_df.show()

+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|   unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|     services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|   management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30|   management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|  blue-collar|married|secondary|     no|      0|    yes|  no| unknown| 

Get the balance of the two youngest people by job


In [10]:
from pyspark.sql.window import Window

byJob = Window.partitionBy("job").orderBy("age")

bank_df \
  .withColumn("new_column_job", row_number().over(byJob)) \
  .filter(col("new_column_job") <= 2) \
  .select("age", "job", "balance") \
  .orderBy("job", "age") \
  .show()

+---+-------------+-------+
|age|          job|balance|
+---+-------------+-------+
| 22|       admin.|   4111|
| 23|       admin.|      5|
| 23|  blue-collar|    817|
| 23|  blue-collar|   8627|
| 23| entrepreneur|      4|
| 25| entrepreneur|  16874|
| 26|    housemaid|    543|
| 26|    housemaid|   -759|
| 23|   management|    736|
| 24|   management|    172|
| 24|      retired|    366|
| 35|      retired|    285|
| 25|self-employed|    453|
| 26|self-employed|    211|
| 21|     services|    361|
| 21|     services|   1903|
| 19|      student|      0|
| 19|      student|    103|
| 22|   technician|    333|
| 23|   technician|    598|
+---+-------------+-------+
only showing top 20 rows



## Exercise 1

Using the dataframe built from `bank.csv`file, get the TOP 3 of maximum balance by marital
Obtén el Top 3 de máximos balances por estado civil


---




In [11]:
byMarital = Window.partitionBy("marital").orderBy(col("balance").desc())

bank_df \
  .withColumn("ranking", row_number().over(byMarital)) \
  .filter(col("ranking") <= 3) \
  .show()

+---+-------------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-------+
|age|          job| marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|ranking|
+---+-------------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-------+
| 54|   management|divorced| tertiary|     no|  26306|    yes|  no|cellular| 11|  feb|      27|       1|   84|       3| failure| no|      1|
| 34|   management|divorced| tertiary|     no|  13204|    yes| yes|cellular| 20|  nov|     197|       2|   -1|       0| unknown| no|      2|
| 51|self-employed|divorced|  primary|     no|  10924|     no|  no|cellular|  6|  may|     106|       2|   -1|       0| unknown| no|      3|
| 60|      retired| married|  primary|     no|  71188|     no|  no|cellular|  6|  oct|     205|       1|   -1|       0| unknown| no|      1|
| 42| entrepr

## Exercise 2



Load `vehicles.csv` file into a DataFrame

---

In [12]:
!head /dataset/vehicles.csv

name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class
Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled
T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft
X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NA,repulsorcraft
TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NA,6.4,1200,1,0,65,2 days,starfighter
Snowspeeder,t-47 airspeeder,Incom corporation,NA,4.5,650,2,0,10,none,airspeeder
TIE bomber,TIE/sa bomber,Sienar Fleet Systems,NA,7.8,850,1,0,none,2 days,space/planetary bomber
AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,20,60,5,40,1000,NA,assault walker
AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,2,90,2,0,200,none,walker
Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin

In [13]:
vehiclesDF_all = spark.read.format("csv")\
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/vehicles.csv")


In [14]:
vehiclesDF_all.show(truncate=False)

+------------------------------+-------------------------------+----------------------------------------------------------+---------------+------+----------------------+----+----------+--------------+---------------+-------------------------+
|name                          |model                          |manufacturer                                              |cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|consumables    |vehicle_class            |
+------------------------------+-------------------------------+----------------------------------------------------------+---------------+------+----------------------+----+----------+--------------+---------------+-------------------------+
|Sand Crawler                  |Digger Crawler                 |Corellia Mining Corporation                               |150000         |36.8  |30                    |46  |30        |50000         |2 months       |wheeled                  |
|T-16 skyhopper             

For each vehicle, get the difference in price (`cost_in_credits`) for each product compared to the cheapest product in the same vehicle class


---



In [15]:
from pyspark.sql.types import IntegerType

vehicles_filtered = vehiclesDF_all \
  .filter(col("cost_in_credits") != "NA") \
  .withColumn("cost_in_credits", col("cost_in_credits").cast(IntegerType()))
vehicles_filtered.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: integer (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: string (nullable = true)
 |-- cargo_capacity: string (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



In [16]:
from pyspark.sql.functions import *

byClass = Window.partitionBy("vehicle_class") \
  .orderBy(col("cost_in_credits").asc())


vehicles_filtered \
  .withColumn("cost_difference", min(col("cost_in_credits")).over(byClass) - col("cost_in_credits")) \
  .withColumn("min_cost", min(col("cost_in_credits")).over(byClass)) \
  .orderBy(col("vehicle_class"), col("cost_in_credits").desc()) \
  .show()

+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+---------------+--------+
|                name|               model|        manufacturer|cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|    consumables|       vehicle_class|cost_difference|min_cost|
+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+---------------+--------+
|Raddaugh Gnasp fl...|Raddaugh Gnasp fl...|Appazanna Enginee...|          14750|     7|                   310|   2|         0|            20|           none|         air speeder|              0|   14750|
|       Droid gunship|   HMP droid gunship|Baktoid Fleet Ord...|          60000|  12.3|                   820|   0|         0|             0|           none|          airspeeder|      

# Joins

## Exercise 3

1. Create dataframes for files `characters.csv` and `planets.csv`
2. Get the planet gravity for each character, selecting only the character name, planet name and gravity.


---




In [17]:
charactersDF_all = spark.read.format("csv")\
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/characters.csv")

In [18]:
charactersDF_all.show(truncate=False)

+---------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|name                 |height|mass |hair_color   |skin_color      |eye_color|birth_year|gender       |homeworld |species       |
+---------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|Luke Skywalker       |172   |77   |blond        |fair            |blue     |19BBY     |male         |Tatooine  |Human         |
|C-3PO                |167   |75   |NA           |gold            |yellow   |112BBY    |NA           |Tatooine  |Droid         |
|R2-D2                |96    |32   |NA           |white, blue     |red      |33BBY     |NA           |Naboo     |Droid         |
|Darth Vader          |202   |136  |none         |white           |yellow   |41.9BBY   |male         |Tatooine  |Human         |
|Leia Organa          |150   |49   |brown        |light           |brown    |19BBY     |female   

In [19]:
planetsDF_all = spark.read.format("csv")\
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/planets.csv")

In [20]:
planetsDF_all.show(truncate=False)

+--------------+---------------+--------------+--------+----------------------+--------------------------------------+----------------------------------------+-------------+-----------+
|name          |rotation_period|orbital_period|diameter|climate               |gravity                               |terrain                                 |surface_water|population |
+--------------+---------------+--------------+--------+----------------------+--------------------------------------+----------------------------------------+-------------+-----------+
|Alderaan      |24             |364           |12500   |temperate             |1 standard                            |grasslands, mountains                   |40           |2000000000 |
|Yavin IV      |24             |4818          |10200   |temperate, tropical   |1 standard                            |jungle, rainforests                     |8            |1000       |
|Hoth          |23             |549           |7200    |frozen        

In [26]:
from pyspark.sql.functions import *

charactersDF_all \
  .join(planetsDF_all, charactersDF_all["homeworld"] == planetsDF_all["name"]) \
  .select(charactersDF_all["name"], planetsDF_all["name"], col("gravity")) \
  .show()

+--------------------+--------------+--------------------+
|                name|          name|             gravity|
+--------------------+--------------+--------------------+
|         Leia Organa|      Alderaan|          1 standard|
| Bail Prestor Organa|      Alderaan|          1 standard|
|     Raymus Antilles|      Alderaan|          1 standard|
|       Ratts Tyerell|   Aleen Minor|                  NA|
|               Lobot|        Bespin|1.5 (surface), 1 ...|
|    Jek Tono Porkins|    Bestine IV|                  NA|
|         Nute Gunray|Cato Neimoidia|          1 standard|
|        Ki-Adi-Mundi|         Cerea|                   1|
|          Mas Amedda|      Champala|                   1|
|          Mon Mothma|     Chandrila|                   1|
|          Jango Fett|  Concord Dawn|                  NA|
|            Han Solo|      Corellia|          1 standard|
|      Wedge Antilles|      Corellia|          1 standard|
|       Finis Valorum|     Coruscant|          1 standar

## Exercise 4

Check exercise 3. What join type are been used? Why?

---

After checking execution plan, execute the following instructions:

---

In [23]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", '0')

In [22]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'10485760b'

Execute again the query of the exercise 3

---

## Exercise 5

1. Create a DataFrame from `species.csv`.
2. Repartition the previous DataFrame to 100 partitions
3. Repartition `characters` DataFrame to 100 partitions

---



In [ ]:
charactersDF_repartition = charactersDF_all.repartition(100)

In [ ]:
speciesDF_all = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/species.csv")

In [ ]:
speciesDF_all.show()

+--------------+--------------+-----------+--------------+--------------------+--------------------+--------------------+----------------+--------------+--------------+
|          name|classification|designation|average_height|         skin_colors|         hair_colors|          eye_colors|average_lifespan|      language|     homeworld|
+--------------+--------------+-----------+--------------+--------------------+--------------------+--------------------+----------------+--------------+--------------+
|          Hutt|     gastropod|   sentient|           300|   green, brown, tan|                  NA|         yellow, red|            1000|       Huttese|     Nal Hutta|
|Yoda's species|        mammal|   sentient|            66|       green, yellow|        brown, white|brown, green, yellow|             900|Galactic basic|            NA|
|    Trandoshan|       reptile|   sentient|           200|        brown, green|                none|      yellow, orange|              NA|          Dosh|  

In [ ]:
speciesDF_repartition = speciesDF_all.repartition(100)

## Exercise 6

Get the specie classification for each character. Select only the character name and its classification<br>
Use DataFrames repartitioned previously


---



In [ ]:
classDF = charactersDF_repartition \
  .join(speciesDF_repartition, charactersDF_repartition["species"] == speciesDF_repartition["name"]) \
  .select(charactersDF_repartition["name"], speciesDF_repartition["classification"]) \


In [ ]:
classDF.show()

+----------------+--------------+
|            name|classification|
+----------------+--------------+
|         Lama Su|     amphibian|
|           IG-88|    artificial|
|      Darth Maul|        mammal|
|         Tarfful|        mammal|
|    Qui-Gon Jinn|        mammal|
|     Bib Fortuna|       mammals|
| Luminara Unduli|        mammal|
| Dexter Jettster|     amphibian|
|Anakin Skywalker|        mammal|
|      Wat Tambor|        mammal|
|          Ackbar|     amphibian|
|           Dormé|        mammal|
|Lando Calrissian|        mammal|
|     Yarael Poof|        mammal|
|    Ki-Adi-Mundi|        mammal|
|        San Hill|        mammal|
|           R2-D2|    artificial|
|  Luke Skywalker|        mammal|
|    Roos Tarpals|     amphibian|
|           Cordé|        mammal|
+----------------+--------------+
only showing top 20 rows



## Exercise 7

1. Execute the following statement over the DataFrame built in exercise 6
2. Check the difference in terms of rows distribution across all partitions

---



In [ ]:
from pyspark.sql.functions import *

classDF \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("count").desc()) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|         65|    1|
|         31|    1|
|         78|    1|
|         34|    1|
|         81|    1|
|         76|    1|
|         28|    1|
|         44|    1|
|         26|    1|
|         27|    1|
|         12|    1|
|         91|    1|
|         93|    1|
|         22|    1|
|         47|    1|
|          1|    1|
|         13|    1|
|          3|    1|
|         20|    1|
|         40|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
classDF \
  .repartition(20) \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("count").desc()) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          8|    6|
|          7|    5|
|          5|    5|
|          1|    5|
|          2|    5|
|         19|    5|
|          6|    5|
|         12|    4|
|         13|    4|
|         10|    4|
|         16|    4|
|          9|    4|
|         17|    4|
|          3|    4|
|         14|    4|
|          0|    3|
|         15|    3|
|         18|    3|
|         11|    3|
|          4|    2|
+-----------+-----+

